In [4]:
# Etapa 1: Query SQL para construir a dimensão de funcionários

# Caminho físico do lakehouse de destino
path_destino = "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/lk_departamento_pessoal.Lakehouse/Tables/tab_silver_dim_funcionario"

from pyspark.sql.functions import col, when, to_date, lit

# 1. Configuração para evitar erro de datas antigas
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")


df_resultado = spark.sql("""
WITH FuncionariosBase AS (
  SELECT
    P.CODCOLIGADA,
    P.CODFILIAL,
    P.CHAPA,
    P.CODPESSOA,
    P.CODSITUACAO,
    PS.DESCRICAO AS DESCRICAO_SITUACAO,
    P.DATAADMISSAO,
    P.TIPOADMISSAO,
    P.DATADEMISSAO,
    P.TIPODEMISSAO,
    P.CODFUNCAO,
    PF.NOME AS DESCRICAO_FUNCAO,
    P.CODSECAO,
    PSC.DESCRICAO AS DESCRICAO_SECAO
  FROM
    pfunc P
    LEFT JOIN pcodsituacao PS ON P.CODSITUACAO = PS.CODCLIENTE
    LEFT JOIN pfuncao PF ON P.CODFUNCAO = PF.CODIGO AND P.CODCOLIGADA = PF.CODCOLIGADA
    LEFT JOIN psecao PSC ON P.CODSECAO = PSC.CODIGO AND P.CODCOLIGADA = PSC.CODCOLIGADA
)

SELECT
  base.CODCOLIGADA,
  base.CODFILIAL,
  base.CHAPA,
  base.CODPESSOA,
  PP.NOME AS NOME_FUNCIONARIO,
  PP.CPF,
  PP.DTNASCIMENTO,
  PP.ESTADOCIVIL,
  PP.SEXO,
  
  CASE 
    WHEN (
      COALESCE(PP.DEFICIENTEAUDITIVO, 0) + 
      COALESCE(PP.DEFICIENTEFALA, 0) + 
      COALESCE(PP.DEFICIENTEFISICO, 0) + 
      COALESCE(PP.DEFICIENTEINTELECTUAL, 0) + 
      COALESCE(PP.DEFICIENTEMENTAL, 0) + 
      COALESCE(PP.DEFICIENTEMOBREDUZIDA, 0) + 
      COALESCE(PP.DEFICIENTEOBSERVACAO, 0) + 
      COALESCE(PP.DEFICIENTEVISUAL, 0)
    ) > 0 THEN 'Sim'
    ELSE 'Não'
  END AS PCD,
  
  base.DATAADMISSAO,
  base.TIPOADMISSAO,
  AD.DESCRICAO AS DESCRICAO_TIPO_ADMISSAO,
  base.DATADEMISSAO,
  base.TIPODEMISSAO,
  DE.DESCRICAO AS DESCRICAO_TIPO_DEMISSAO,
  base.DESCRICAO_SITUACAO,
  base.DESCRICAO_FUNCAO,
  base.DESCRICAO_SECAO,
  PCI.DESCRICAO AS DESCRICAO_GRAU_INSTRUCAO,
  PN.DESCRICAO AS DESCRICAO_NACIONALIDADE
FROM
  FuncionariosBase base
  LEFT JOIN ppessoa PP ON base.CODPESSOA = PP.CODIGO
  LEFT JOIN pcodinstrucao PCI ON PP.GRAUINSTRUCAO = PCI.CODCLIENTE
  LEFT JOIN pcodnacao PN ON PP.NACIONALIDADE = PN.CODCLIENTE
  LEFT JOIN PTPADMISSAO AD ON base.TIPOADMISSAO = AD.CODCLIENTE
  LEFT JOIN PTPDEMISSAO DE ON base.TIPODEMISSAO = DE.CODCLIENTE
""")

# 3. Corrigir datas inválidas (< 1900-01-01)
data_minima = to_date(lit("1900-01-01"))
data_padrao = to_date(lit("2000-01-01"))

df_corrigido = df_resultado \
    .withColumn("DTNASCIMENTO", when(col("DTNASCIMENTO") < data_minima, data_padrao).otherwise(col("DTNASCIMENTO"))) \
    .withColumn("DATAADMISSAO", when(col("DATAADMISSAO") < data_minima, data_padrao).otherwise(col("DATAADMISSAO"))) \
    .withColumn("DATADEMISSAO", when(col("DATADEMISSAO") < data_minima, data_padrao).otherwise(col("DATADEMISSAO")))


# Salva no Lakehouse de destino usando caminho físico
df_corrigido.write.format("delta").mode("overwrite").save(path_destino)

StatementMeta(, 56785b0a-e1a3-4c1c-932e-1991afbd8d9a, 6, Finished, Available, Finished)